In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from experiments.extract import extract
from experiments.experiment import Experiment
from experiments.polar import LatLonPolar

import pandas as pd
import altair as alt
from lightgbm import LGBMRegressor

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder
from category_encoders import TargetEncoder, QuantileEncoder

pd.options.display.float_format = '{:,.2f}'.format

# Data Sample

In [6]:
X = extract('SALE')
X['month'] = pd.to_datetime(X.created_date.dt.strftime('%Y-%m-01'))
print(X.shape)
X.sample(5).T

(4985, 14)


title,"Casa de Condomínio com dois quartos , na Rua Barão de Piraquara - Realengo.",Apartamento-À VENDA-Praça Seca-Rio de Janeiro-RJ,"Apartamento à venda, 62 m² por R$ 235.000,00 - Praça Seca - Rio de Janeiro/RJ",REALENGO - EXCLUSIVIDADE. MARAVILHOSO IMÓVEL DUPLEX EM CONDOMINIO,"Praça Seca | Apartamento 3 quartos, sendo 1 suite"
url,/imovel/venda-casa-de-condominio-2-quartos-com-cozinha-realengo-zona-oeste-rio-de-janeiro-rj-140m2-id-2595453461/,/imovel/venda-apartamento-2-quartos-com-churrasqueira-praca-seca-zona-oeste-rio-de-janeiro-rj-52m2-id-2582558660/,/imovel/apartamento-2-quartos-praca-seca-zona-oeste-rio-de-janeiro-com-garagem-62m2-venda-RS235000-id-2574150606/,/imovel/venda-casa-4-quartos-com-piscina-realengo-zona-oeste-rio-de-janeiro-rj-180m2-id-2586622319/,/imovel/apartamento-3-quartos-praca-seca-zona-oeste-rio-de-janeiro-com-garagem-116m2-venda-RS400000-id-2646784776/
origin,zapimoveis,zapimoveis,vivareal,zapimoveis,vivareal
neighborhood,Realengo,Praça Seca,Praça Seca,Realengo,Praça Seca
usable_area,140,52,62,180,116
unit_types,HOME,APARTMENT,APARTMENT,HOME,APARTMENT
floors,0,0,0,0,6
bedrooms,2,2,2,4,3
bathrooms,2,1,2,5,3
suites,0,0,1,3,1
parking_spaces,0,1,1,3,2


# Simple Model

In [7]:
y = X.pop('total')
exp = Experiment(X, y, 5)
exp

Experiment(n_splits=5, metrics=[<function mean_absolute_error at 0x7fbf234ee830>, <function rmse at 0x7fbf230136d0>, <function mean_absolute_percentage_error at 0x7fbf234ee950>])

In [8]:
simple_pipe = make_column_transformer(
    ('passthrough', ['usable_area', 'bedrooms', 'bathrooms', 'parking_spaces'])
)
exp_simple_pipe = exp.run('simple', LGBMRegressor(), {}, simple_pipe)


simple_pipe_unit = make_column_transformer(
    ('passthrough', ['usable_area', 'bedrooms', 'bathrooms', 'parking_spaces']),
    (OneHotEncoder(), ['unit_types'])
)
exp_pipe_unit = exp.run('unit', LGBMRegressor(), {}, simple_pipe_unit)

simple_pipe_latlon = make_column_transformer(
    ('passthrough', ['usable_area', 'bedrooms', 'bathrooms', 'parking_spaces', 'lat', 'lon']),
    (OneHotEncoder(), ['unit_types'])
)
exp_pipe_latlon = exp.run('latlng', LGBMRegressor(), {}, simple_pipe_latlon)

simple_pipe_neighborhood = make_column_transformer(
    ('passthrough', ['usable_area', 'bedrooms', 'bathrooms', 'parking_spaces', 'lat', 'lon']),
    (OneHotEncoder(handle_unknown='ignore'), ['neighborhood', 'unit_types'])
)
exp_pipe_neighborhood = exp.run('neighborhood', LGBMRegressor(), {}, simple_pipe_neighborhood)

simple_pipe_target = make_column_transformer(
    ('passthrough', ['usable_area', 'bedrooms', 'bathrooms', 'parking_spaces', 'lat', 'lon']),
    (TargetEncoder(), ['neighborhood', 'unit_types'])
)
exp_pipe_target = exp.run('neighborhood_target', LGBMRegressor(), {}, simple_pipe_target)

simple_pipe_quantile = make_column_transformer(
    ('passthrough', ['usable_area', 'bedrooms', 'bathrooms', 'parking_spaces', 'lat', 'lon']),
    (QuantileEncoder(), ['neighborhood', 'unit_types'])
)
exp_pipe_quantile = exp.run('neighborhood_quantile', LGBMRegressor(), {}, simple_pipe_quantile)

exp.obs_metrics.query('split_name == "test"').groupby(['name', 'metric']).value.mean().unstack().sort_values('mae')

metric,mae,mape,rmse
name,,,
unit,"181,365.40",0.66,"2,168,224.99"
simple,"182,350.53",0.66,"2,169,298.40"
neighborhood,"200,665.21",0.88,"2,414,209.80"
latlng,"200,977.56",0.89,"2,416,006.77"
neighborhood_quantile,"201,282.70",0.89,"2,419,201.18"
neighborhood_target,"201,396.43",0.89,"2,416,428.03"


In [9]:
polar_target = make_column_transformer(
    ('passthrough', ['usable_area', 'bedrooms', 'bathrooms', 'parking_spaces']),
    (LatLonPolar('neighborhood'), ['neighborhood', 'lat', 'lon']),
    (TargetEncoder(), ['neighborhood', 'unit_types'])
)
exp_pipe_target = exp.run('polar_target', LGBMRegressor(), {}, polar_target)

exp.obs_metrics.query('split_name == "test"').groupby(['name', 'metric']).value.mean().unstack().sort_values('mae')

metric,mae,mape,rmse
name,,,
unit,"181,365.40",0.66,"2,168,224.99"
simple,"182,350.53",0.66,"2,169,298.40"
neighborhood,"200,665.21",0.88,"2,414,209.80"
latlng,"200,977.56",0.89,"2,416,006.77"
neighborhood_quantile,"201,282.70",0.89,"2,419,201.18"
neighborhood_target,"201,396.43",0.89,"2,416,428.03"
polar_target,"217,857.49",0.92,"2,421,334.28"


In [10]:
def clip_target(X, y, quantiles=[.05, .95]):
    qinf = y.quantile(quantiles[0])
    qsup = y.quantile(quantiles[1])
    return X, y.clip(lower=qinf, upper=qsup)

exp_clipped_latlon = exp.run('clipped latlng', LGBMRegressor(), {}, simple_pipe_target, preprocess_train_fn=clip_target)

exp_clipped_neighborhood = exp.run('clipped neighborhood', LGBMRegressor(), {}, simple_pipe_neighborhood, preprocess_train_fn=clip_target)

exp.obs_metrics.query('split_name == "test"').groupby(['name', 'metric']).value.median().unstack().sort_values('mae')

metric,mae,mape,rmse
name,,,
clipped latlng,"70,626.24",0.21,"193,740.23"
clipped neighborhood,"70,968.62",0.21,"194,414.03"
unit,"141,477.63",0.71,"645,289.27"
simple,"142,768.17",0.71,"649,973.43"
neighborhood_target,"209,686.09",0.75,"1,841,526.47"
latlng,"209,789.91",0.76,"1,842,615.03"
neighborhood,"210,968.54",0.73,"1,830,612.07"
neighborhood_quantile,"211,128.55",0.74,"1,843,577.65"
polar_target,"224,621.71",0.85,"1,443,477.95"


In [11]:
base = alt.Chart(
    exp.obs_metrics.assign(name = exp.obs_metrics.metric + ' | ' + exp.obs_metrics.name + ' | ' + exp.obs_metrics.split_name).query('metric == "mape"')
)

selection = alt.selection_multi(fields=['name'], bind='legend')
opacity = alt.condition(selection, alt.value(1.0), alt.value(0))

(
    base.mark_point().encode(x = 'split', y = 'value', color = 'name', tooltip = ['name', 'value'], opacity = opacity) + 
    base.mark_line().encode(x = 'split', y = 'value', color = 'name', opacity = opacity)
).add_selection(selection).properties(width=900, height=600).interactive(bind_x = False)

alt.LayerChart(...)

In [12]:
y_pred = exp_clipped_latlon.predict(X, y)
points = [0] + [i.right for i in pd.qcut(y, 15).cat.categories.values]

pd.DataFrame({
    'true': pd.cut(y, points).tolist(),
    'pred': pd.cut(y_pred, points).tolist()
}).groupby(['true', 'pred'])\
.value_counts().unstack().fillna(0).astype(int)\
.style.background_gradient(axis=1)

pred,"(0.0, 120250.0]","(120250.0, 140380.0]","(140380.0, 159840.0]","(159840.0, 175233.0]","(175233.0, 190110.0]","(190110.0, 200677.8]","(200677.8, 220697.867]","(220697.867, 240310.667]","(240310.667, 260000.0]","(260000.0, 290010.333]","(290010.333, 324230.133]","(324230.133, 369000.0]","(369000.0, 420493.333]","(420493.333, 550000.0]","(550000.0, 145000000.0]"
true,,,,,,,,,,,,,,,
"(0.0, 120250.0]",56,102,70,31,26,9,25,7,6,0,1,1,1,0,0
"(120250.0, 140380.0]",12,74,110,57,34,8,15,12,4,4,2,0,1,0,0
"(140380.0, 159840.0]",4,40,79,70,33,15,44,14,15,7,5,1,3,0,0
"(159840.0, 175233.0]",1,13,40,69,56,26,69,29,17,9,1,1,1,0,0
"(175233.0, 190110.0]",0,5,29,36,39,27,106,52,20,13,2,1,2,0,0
"(190110.0, 200677.8]",0,5,18,37,35,42,106,36,34,11,4,1,1,1,1
"(200677.8, 220697.867]",0,4,14,22,28,29,79,73,36,26,15,0,5,1,0
"(220697.867, 240310.667]",0,1,5,9,17,13,73,76,68,48,11,10,2,0,0
"(240310.667, 260000.0]",0,1,0,1,14,18,73,56,83,59,16,11,1,4,0


In [15]:
pd.DataFrame({
    'feature': sum(exp_clipped_latlon.model[:-1][0]._columns, []),
    'imp': exp_clipped_latlon.model._final_estimator.feature_importances_
}).sort_values('imp', ascending=False)

,feature,imp
0,usable_area,1190
5,lon,514
4,lat,499
3,parking_spaces,236
2,bathrooms,211
1,bedrooms,199
7,unit_types,78
6,neighborhood,73


In [14]:
import m2cgen as m2c

code = m2c.export_to_go(exp_clipped_neighborhood.model._final_estimator, function_name='Predict')
code

'func Predict(input []float64) float64 {\n    var var0 float64\n    if input[0] > 102.50000000000001 {\n        if input[0] > 144.50000000000003 {\n            if input[0] > 316.00000000000006 {\n                var0 = 291794.9429328361\n            } else {\n                if input[1] > 2.5000000000000004 {\n                    if input[4] <= -22.901590347290036 {\n                        var0 = 272040.43401466217\n                    } else {\n                        if input[0] > 145.50000000000003 {\n                            if input[0] > 151.00000000000003 {\n                                if input[0] > 156.50000000000003 {\n                                    if input[3] > 2.5000000000000004 {\n                                        var0 = 288629.3462768541\n                                    } else {\n                                        var0 = 283681.7121321369\n                                    }\n                                } else {\n                          